In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re


from PIL import Image
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import pos_tag
import collections
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

import string
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.dummy import DummyClassifier

In [2]:
# pip install emoji --upgrade

In [3]:
import emoji

In [4]:
sw = list(stopwords.words('english'))

In [5]:
tweets = pd.read_csv('Data/Tweets.csv')

In [6]:
tweets.isna().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [7]:
tweets['negativereason_gold'].value_counts()

Customer Service Issue                      12
Late Flight                                  4
Can't Tell                                   3
Cancelled Flight                             3
Cancelled Flight\nCustomer Service Issue     2
Late Flight\nLost Luggage                    1
Customer Service Issue\nLost Luggage         1
Late Flight\nFlight Attendant Complaints     1
Flight Attendant Complaints                  1
Lost Luggage\nDamaged Luggage                1
Customer Service Issue\nCan't Tell           1
Bad Flight                                   1
Late Flight\nCancelled Flight                1
Name: negativereason_gold, dtype: int64

In [8]:
tweets['airline_sentiment_gold'].value_counts()

negative    32
positive     5
neutral      3
Name: airline_sentiment_gold, dtype: int64

In [9]:
tweets['retweet_count'].value_counts()

0     13873
1       640
2        66
3        22
4        17
5         5
7         3
6         3
22        2
8         1
32        1
9         1
31        1
18        1
15        1
28        1
44        1
11        1
Name: retweet_count, dtype: int64

In [10]:
tweets['tweet_created'].value_counts()

2015-02-24 09:54:34 -0800    5
2015-02-24 11:43:05 -0800    4
2015-02-24 09:14:01 -0800    3
2015-02-24 11:32:49 -0800    3
2015-02-23 06:57:24 -0800    3
                            ..
2015-02-22 17:45:23 -0800    1
2015-02-18 11:15:40 -0800    1
2015-02-18 00:49:34 -0800    1
2015-02-23 04:42:51 -0800    1
2015-02-23 13:10:14 -0800    1
Name: tweet_created, Length: 14247, dtype: int64

In [11]:
tweets['user_timezone'].value_counts()

Eastern Time (US & Canada)    3744
Central Time (US & Canada)    1931
Pacific Time (US & Canada)    1208
Quito                          738
Atlantic Time (Canada)         497
                              ... 
Monterrey                        1
Newfoundland                     1
Saskatchewan                     1
Wellington                       1
Midway Island                    1
Name: user_timezone, Length: 85, dtype: int64

In [12]:
tweets['name'].value_counts()

JetBlueNews       63
kbosspotter       32
_mhertz           29
otisday           28
throthra          27
                  ..
MarriedMariner     1
mikejmarsh         1
Clking_sf          1
MsGeekyTeach       1
CoralReefer420     1
Name: name, Length: 7701, dtype: int64

In [13]:
tweets.drop('negativereason_gold', axis=1, inplace=True)
tweets.drop('airline_sentiment_gold', axis=1, inplace=True)
tweets.drop('retweet_count', axis=1, inplace=True)
tweets.drop('tweet_created', axis=1, inplace=True)
tweets.drop('user_timezone', axis=1, inplace=True)
tweets.drop('name', axis=1, inplace=True)

#current list of columns
list(tweets.columns)

['tweet_id',
 'airline_sentiment',
 'airline_sentiment_confidence',
 'negativereason',
 'negativereason_confidence',
 'airline',
 'text',
 'tweet_coord',
 'tweet_location']

In [14]:
tweets['airline_sentiment_confidence'].value_counts()

1.0000    10445
0.6667       71
0.6632       35
0.6559       30
0.6596       30
          ...  
0.3913        1
0.7273        1
0.6353        1
0.6260        1
0.3544        1
Name: airline_sentiment_confidence, Length: 1023, dtype: int64

In [15]:
tweets['negativereason'].value_counts()

Customer Service Issue         2910
Late Flight                    1665
Can't Tell                     1190
Cancelled Flight                847
Lost Luggage                    724
Bad Flight                      580
Flight Booking Problems         529
Flight Attendant Complaints     481
longlines                       178
Damaged Luggage                  74
Name: negativereason, dtype: int64

In [16]:
tweets['negativereason_confidence'].value_counts()

1.0000    3436
0.0000    1344
0.6667      62
0.6632      33
0.6596      29
          ... 
0.3386       1
0.3577       1
0.3249       1
0.3290       1
0.3255       1
Name: negativereason_confidence, Length: 1410, dtype: int64

In [17]:
tweets['airline'].value_counts()

United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: airline, dtype: int64

In [18]:
tweets['tweet_coord'].value_counts()

[0.0, 0.0]                     164
[40.64656067, -73.78334045]      6
[32.91792297, -97.00367737]      3
[40.64646912, -73.79133606]      3
[40.68994668, -73.91637642]      2
                              ... 
[39.60813937, -105.9548521]      1
[39.1780768, -76.670155]         1
[41.71826065, -72.68689133]      1
[35.99558298, -78.89974547]      1
[29.98521945, -90.25215941]      1
Name: tweet_coord, Length: 832, dtype: int64

In [19]:
tweets['tweet_location'].value_counts()

Boston, MA                   157
New York, NY                 156
Washington, DC               150
New York                     127
USA                          126
                            ... 
Roseville, California          1
Dirty South Jerz               1
Hershey, PA                    1
Born in NY, living in PA       1
Missouri State University      1
Name: tweet_location, Length: 3081, dtype: int64

In [20]:
# tweets.drop('tweet_id', axis=1, inplace=True)
# tweets['name'].value_counts()
# tweets['name'].value_counts()
# tweets['name'].value_counts()

In [21]:
tweets = tweets[tweets['airline_sentiment_confidence'] > 0.80]

In [22]:
tweets.shape

(10459, 9)

In [23]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10459 entries, 0 to 14638
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      10459 non-null  int64  
 1   airline_sentiment             10459 non-null  object 
 2   airline_sentiment_confidence  10459 non-null  float64
 3   negativereason                7392 non-null   object 
 4   negativereason_confidence     7396 non-null   float64
 5   airline                       10459 non-null  object 
 6   text                          10459 non-null  object 
 7   tweet_coord                   746 non-null    object 
 8   tweet_location                7007 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 817.1+ KB


In [24]:
#just in case
def labels_to_num(label):
    if label == 'negative':
        label = 0
    elif label == 'positive':
        label = 1
    else:
        label = 2
    return label

In [25]:
tweets['airline_sentiment_num'] = tweets['airline_sentiment'].map(labels_to_num)

In [26]:
tweets['airline_sentiment_num'].value_counts()

0    7392
2    1550
1    1517
Name: airline_sentiment_num, dtype: int64

In [27]:
##Translate nltk POS to wordnet tags for lemmatizer
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [28]:
# from nltk.stem import PorterStemmer

In [29]:
from nltk.tokenize import word_tokenize

In [30]:
lemmatizer = WordNetLemmatizer() #lemmatizing
#STOPWORDS

def process_text(text):
    doc = text.lower()
    doc = re.sub(r'@\w+', '', doc) # Remove @s
    doc = re.sub(r'#', '', doc) # Remove hashtags
    doc = re.sub(r':', ' ', emoji.demojize(doc)) # Turn emojis into words
    doc = re.sub(r'http\S+', '',doc) # Remove URLs
    doc = re.sub(r'\$\S+', 'dollar', doc) # Change dollar amounts to dollar
    doc = re.sub(r'[^a-z0-9\s]', '', doc) # Remove punctuation
    doc = re.sub(r'[0-9]+', 'number', doc) # Change number values to number
    
    doc = word_tokenize(doc)
#     doc = [word for word in doc if word not in sw]
   
    
    doc = pos_tag(doc)

    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]

    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc] 

    return ' '.join(doc)

In [31]:
tweets['processed_text'] = tweets['text'].map(process_text)

In [32]:
tweets['text'][0]

'@VirginAmerica What @dhepburn said.'

In [33]:
tweets['processed_text'][0]

'what say'

In [34]:
dummied_airlines = pd.get_dummies(tweets['airline'])

In [35]:
tweets = tweets.join(dummied_airlines)

In [36]:
tweets.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline', 'text',
       'tweet_coord', 'tweet_location', 'airline_sentiment_num',
       'processed_text', 'American', 'Delta', 'Southwest', 'US Airways',
       'United', 'Virgin America'],
      dtype='object')

## Vectorizing

In [87]:
X = tweets.drop(columns=['tweet_id', 'airline_sentiment_confidence', 'airline',
       'negativereason', 'negativereason_confidence', 'text', 'tweet_coord', 'tweet_location', 'airline_sentiment_num', 'airline_sentiment'], axis =1)
y = tweets['airline_sentiment']

In [92]:
##running our train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.20)

In [89]:
#try different parameters
vectorizer = TfidfVectorizer(input='content')

In [93]:
sparse_out = vectorizer.fit_transform(X_train['processed_text'])
tfidf_df_train = pd.DataFrame(data = sparse_out.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_train.index)

X_train_vec = X_train.drop(['processed_text'], axis=1,).join(tfidf_df_train)

In [94]:
sparse_out1 = vectorizer.transform(X_test['processed_text'])
tfidf_df_test = pd.DataFrame(data = sparse_out1.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_test.index)

X_test_vec = X_test.drop(['processed_text'], axis=1).join(tfidf_df_test)
X_test_vec = X_test_vec.fillna(value=0)

In [42]:
#smoteN

In [43]:
tweets.airline_sentiment.value_counts()

negative    7392
neutral     1550
positive    1517
Name: airline_sentiment, dtype: int64

In [95]:
X_train_vec.shape

(8367, 8782)

In [96]:
X_test_vec.shape

(2092, 8782)

### Dummy classifier

In [46]:
##checking our dummy/base model score
dc = DummyClassifier(strategy='stratified')
dc.fit(X_train_vec, y_train)

print(cross_val_score(dc, X_train_vec, y_train).mean())

0.5326885433917422


### Support Vector Classifier

In [47]:
svc = svm.SVC() #Grid search ()
svc.fit(X_train_vec, y_train)

print(cross_val_score(svc, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:  9.8min remaining: 14.6min
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:  9.8min finished


0.8355443579630378


In [50]:
from sklearn.metrics import confusion_matrix

In [52]:
svc_pred_train= svc.predict(X_train_vec)

In [53]:
accuracy_score(y_train, svc_pred_train)

0.9500418310027489

### Logistic Regression|

In [54]:
from sklearn.linear_model import LogisticRegression

In [97]:
lr = LogisticRegression(max_iter=150) #max iterations-change; l2 or l1 - Grid search (l1, l2, C=[.1, 1, 10])  # Suppose .1 is best, then for Round #2 C=[.03, .1, .3
lr.fit(X_train_vec, y_train)
print(cross_val_score(lr, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:   38.7s remaining:   58.1s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:   40.4s finished


0.865543408167244


In [99]:
lr_pred_train= lr.predict(X_train_vec)

In [101]:
accuracy_score(y_train, lr_pred_train)

0.9254212979562567

### Naive Bayes

In [59]:
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
print(cross_val_score(nb, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:    1.8s remaining:    2.8s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:    1.9s finished


0.7422002126685621


### Vectorizing parameters change

In [103]:
#trying different parameters
vectorizer = TfidfVectorizer(input='content', stop_words=sw)
sparse_out = vectorizer.fit_transform(X_train['processed_text'])
tfidf_df_train = pd.DataFrame(data = sparse_out.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_train.index)

X_train_vec = X_train.drop(['processed_text'], axis=1,).join(tfidf_df_train)

In [104]:
lr1 = LogisticRegression(max_iter=150) #max iterations-change; l2 or l1 - Grid search (l1, l2, C=[.1, 1, 10])  # Suppose .1 is best, then for Round #2 C=[.03, .1, .3
lr1.fit(X_train_vec, y_train)
print(cross_val_score(lr1, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:   39.2s remaining:   58.8s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:   40.4s finished


0.8479746140294122


In [105]:
lr1_pred_train= lr1.predict(X_train_vec)

In [106]:
accuracy_score(y_train, lr1_pred_train)

0.9163379945022111

In [238]:
vectorizer = TfidfVectorizer(input='content', max_df=2000, min_df=10)
sparse_out = vectorizer.fit_transform(X_train['processed_text'])
tfidf_df_train = pd.DataFrame(data = sparse_out.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_train.index)

X_train_vec = X_train.drop(['processed_text'], axis=1,).join(tfidf_df_train)

In [239]:
lr2 = LogisticRegression(max_iter=150) #max iterations-change; l2 or l1 - Grid search (l1, l2, C=[.1, 1, 10])  # Suppose .1 is best, then for Round #2 C=[.03, .1, .3
lr2.fit(X_train_vec, y_train)
print(cross_val_score(lr2, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:    4.2s remaining:    6.3s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:    4.5s finished


0.8725949635114164


In [240]:
lr2_pred_train= lr2.predict(X_train_vec)

In [241]:
accuracy_score(y_train, lr2_pred_train) #less overfit and more accurate now

0.9071351738974542

In [204]:
vectorizer = TfidfVectorizer(input='content', max_df=2000, min_df=10, ngram_range=(1, 3))
sparse_out = vectorizer.fit_transform(X_train['processed_text'])
tfidf_df_train = pd.DataFrame(data = sparse_out.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_train.index)

X_train_vec = X_train.drop(['processed_text'], axis=1,).join(tfidf_df_train)

In [211]:
lr3 = LogisticRegression(max_iter=150) #max iterations-change; l2 or l1 - Grid search (l1, l2, C=[.1, 1, 10])  # Suppose .1 is best, then for Round #2 C=[.03, .1, .3
lr3.fit(X_train_vec, y_train)
print(cross_val_score(lr3, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.


0.8206051413231868


[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:    0.4s remaining:    0.7s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:    0.5s finished


In [212]:
lr3_pred_train= lr3.predict(X_train_vec)

In [213]:
accuracy_score(y_train, lr3_pred_train) #no positive change

0.8265806143181547

### Logistic regression gridsearch

In [243]:
from sklearn.model_selection import GridSearchCV

In [284]:
vectorizer = TfidfVectorizer(input='content', max_df=2000, min_df=10)
sparse_out = vectorizer.fit_transform(X_train['processed_text'])
tfidf_df_train = pd.DataFrame(data = sparse_out.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_train.index)

X_train_vec = X_train.drop(['processed_text'], axis=1,).join(tfidf_df_train)

In [285]:
lr2 = LogisticRegression(max_iter=150) #max iterations-change; l2 or l1 - Grid search (l1, l2, C=[.1, 1, 10])  # Suppose .1 is best, then for Round #2 C=[.03, .1, .3
lr2.fit(X_train_vec, y_train)
print(cross_val_score(lr2, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:    4.2s remaining:    6.3s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:    4.3s finished


0.8725949635114164


In [286]:
lr2_pred_train= lr2.predict(X_train_vec)

In [287]:
accuracy_score(y_train, lr2_pred_train) #less overfit and more accurate now

0.9071351738974542

In [288]:
#max iterations-change; l2 or l1 - Grid search (l1, l2, C=[.1, 1, 10])  # Suppose .1 is best, then for Round #2 C=[.03, .1, .3

parameters = {'penalty':['l1', 'l2', 'elasticnet'], 'C': [.1, 1, 10], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

lrgrid1 = GridSearchCV(estimator=lr2, param_grid=parameters, cv=5, n_jobs=-4, verbose=3)
lrgrid1.fit(X_train_vec, y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done  34 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-4)]: Done 138 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-4)]: Done 225 out of 225 | elapsed:  2.7min finished


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=150), n_jobs=-4,
             param_grid={'C': [0.1, 1, 10],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             verbose=3)

In [289]:
lrgrid.best_params_

{'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}

In [290]:
lrgrid.best_score_

0.8740287980941239

In [291]:
lr2grid1_pred_train= lrgrid.best_estimator_.predict(X_train_vec)

In [292]:
accuracy_score(y_train, lr2grid1_pred_train) 

0.9364168758216804

In [293]:
parameters = {'penalty':['l1', 'l2', 'elasticnet'], 'C': [5, 10, 15], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

lrgrid2 = GridSearchCV(estimator=lr2, param_grid=parameters, cv=5, n_jobs=-4, verbose=3)
lrgrid2.fit(X_train_vec, y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done  34 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-4)]: Done 133 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-4)]: Done 225 out of 225 | elapsed:  4.9min finished


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=150), n_jobs=-4,
             param_grid={'C': [5, 10, 15],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             verbose=3)

In [294]:
lrgrid2.best_params_

{'C': 5, 'penalty': 'l2', 'solver': 'liblinear'}

In [295]:
lrgrid2.best_score_

0.8783315872801634

In [296]:
lr2grid2_pred_train= lrgrid2.best_estimator_.predict(X_train_vec)

In [297]:
accuracy_score(y_train, lr2grid2_pred_train) #less overfit and more accurate now

0.9269750209155013

In [298]:
parameters = {'penalty':['l2'], 'C': [3, 4, 4.5, 5, 5.5, 7, 10, 12], 'solver': ['liblinear']}

lrgrid3 = GridSearchCV(estimator=lr2, param_grid=parameters, cv=5, n_jobs=-4, verbose=3)
lrgrid3.fit(X_train_vec, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done  22 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-4)]: Done  40 out of  40 | elapsed:    3.0s finished


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=150), n_jobs=-4,
             param_grid={'C': [3, 4, 4.5, 5, 5.5, 7, 10, 12], 'penalty': ['l2'],
                         'solver': ['liblinear']},
             verbose=3)

In [299]:
lrgrid3.best_params_

{'C': 5.5, 'penalty': 'l2', 'solver': 'liblinear'}

In [300]:
lrgrid3.best_score_

0.8784509901799684

In [301]:
lr2grid3_pred_train = lrgrid3.predict(X_train_vec)

In [302]:
accuracy_score(y_train, lr2grid3_pred_train) #less overfit and more accurate now

0.9287677781761683

### Required Configurations and Python Libraries

In [46]:
# pip install tweepy

In [370]:
# import tweepy as tw
# import json
# import time
# import datetime
# import os
# import csv
# import pickle

In [371]:
Twitter={}
Twitter['Consumer Key'] = 'iAimqbAdWiBrUfXe0Xmuhlyps'
Twitter['Consumer Secret'] = 'YWUyAsX8sA2VuJCwQv4x3MUhAGmm48VlV0ZTx2yWNWqlGCP9Kj'
Twitter['Access Token'] = '54188780-kRmlqKRHleSb1WBBTfrj4CR57wLxJYuU5qH9piYlq'
Twitter['Access Token Secret'] = 'JXzf6VKWIFhVhAuIYWWQX04jCaHgfgjia77ARpK7v1Z3p'

In [318]:
!pip install twitter

In [372]:
import twitter

auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)

In [374]:
airlines_list = tweets.columns[-6:].tolist()
airlines_dict = {i: '#'+i.replace(' ', '') for i in airlines_list }

In [377]:
airlines_dict

{'American': '#AmericanAirlines',
 'Delta': '#DeltaAirlines',
 'Southwest': '#SouthwestAirlines',
 'US Airways': '#USAirways',
 'United': '#UnitedAirlines',
 'Virgin America': '#VirginAmerica'}

In [376]:
airlines_dict['American'] = '#AmericanAirlines'
airlines_dict['United'] = '#UnitedAirlines'
airlines_dict['Southwest'] = '#SouthwestAirlines'
airlines_dict['Delta'] = '#DeltaAirlines'

In [446]:
def get_tweets(hashtag, number):
    list_of_tweets =[]
    list_of_dirty_tweets =[]
    times = []
    search_results = twitter_api.search.tweets(q=hashtag, count=number)
#     print(hashtag + ':' + str(len(search_results['statuses'])) + "\n")
    for num in range(len(search_results['statuses'])):
        list_of_tweets.append(process_text(search_results['statuses'][num]['text']))
        list_of_dirty_tweets.append(search_results['statuses'][num]['text'])
        times.append(search_results['statuses'][num]['user']['created_at'])
    return list_of_tweets, list_of_dirty_tweets, times

In [447]:
X_real_by_airline = {}
for column_name, hashtag  in airlines_dict.items():
    X_real_one_airline = pd.DataFrame(get_tweets(hashtag, 1000)[0], columns=['processed_text'])
    X_real_one_airline['dirty_text'] = get_tweets(hashtag, 1000)[1]
    X_real_one_airline['time_of_tweet'] = get_tweets(hashtag, 1000)[2]
    for other_col_name in airlines_dict.keys():
        if other_col_name == column_name:
            X_real_one_airline[column_name] = 1
        else:
            X_real_one_airline[other_col_name] = 0
            
    X_real_by_airline[column_name] = X_real_one_airline
    
    
#X_real = pd.DataFrame(get_tweets('#DeltaAirlines', 7), columns=['processed_text'])

In [448]:
X_real_by_airline['United']

,processed_text,dirty_text,time_of_tweet,American,Delta,Southwest,US Airways,United,Virgin America
0,rt do you know that our employee and their family can travel the world with flight privilege on united airline and it gl,RT @UGESocial: Did you know that our employees and their families can travel the world with flight privileges on United Airlines and its gl…,Sat Mar 16 21:12:22 +0000 2013,0,0,0,0,1,0
1,dear why fly when you lose people bag unitedairlines lose bag and do nothing about it,"Dear @united, why fly when you lose people’s bags? #unitedairlines lose bags and do nothing about it… @CNTraveler @cntraveller",Sun Nov 22 14:42:12 +0000 2009,0,0,0,0,1,0
2,do you know that our employee and their family can travel the world with flight privilege on united airline a tcoxidckurzgm,Did you know that our employees and their families can travel the world with flight privileges on United Airlines a… https://t.co/xIDCKURZgm,Wed Jul 06 04:15:12 +0000 2022,0,0,0,0,1,0
3,icao anumberd flt ualnumber unitedairlines bosiah first see number number number number min alt number ft msl min dist tcoircoirmnumbera,ICAO: #A4460D\nFlt: #UAL560 #UnitedAirlines #BOS-#IAH\nFirst seen: 2022/08/19 17:53:04\nMin Alt: 1800 ft MSL\nMin Dist:… https://t.co/IrCOiRM01A,Thu Apr 22 13:41:25 +0000 2021,0,0,0,0,1,0
4,one of the best game ever speaknoevilmonkey newmobilegame leagueofangels unitedairlines spring tcogyjhnumberaunnumbery,One of the best games ever.🙊 #NewMobileGame #Leagueofangels #UnitedAirlines #spring https://t.co/GyJh7aun3Y,Sun Jun 26 17:36:51 +0000 2022,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
95,rt barcelona and the boeing bnumber americanairlines unitedairlines emirate avgeeks avgeek aviationphotography,RT @CAlloing: #Barcelona and the #Boeing #B772 ! #AmericanAirlines #UnitedAirlines #Emirates #avgeeks #Avgeek #aviationphotography https:/…,Mon Oct 03 22:02:26 +0000 2011,0,0,0,0,1,0
96,icao anumberanumber flt ualnumber unitedairlines iahlhr first see number number number number min alt number ft msl min dist number tcoqrozorycpe,ICAO: A31A50\nFlt: UAL5 #UnitedAirlines #IAH-#LHR\nFirst seen: 2022/08/19 10:30:26\nMin Alt: 23600 ft MSL\nMin Dist: 3.… https://t.co/qrozoRYCpE,Wed Mar 30 16:31:03 +0000 2022,0,0,0,0,1,0
97,rt barcelona and the boeing bnumber americanairlines unitedairlines emirate avgeeks avgeek aviationphotography,RT @CAlloing: #Barcelona and the #Boeing #B772 ! #AmericanAirlines #UnitedAirlines #Emirates #avgeeks #Avgeek #aviationphotography https:/…,Fri Jun 14 08:20:09 +0000 2019,0,0,0,0,1,0
98,rt barcelona and the boeing bnumber americanairlines unitedairlines emirate avgeeks avgeek aviationphotography,RT @CAlloing: #Barcelona and the #Boeing #B772 ! #AmericanAirlines #UnitedAirlines #Emirates #avgeeks #Avgeek #aviationphotography https:/…,Sat May 02 14:48:28 +0000 2020,0,0,0,0,1,0


In [451]:
sparse_out2 = vectorizer.transform(X_real_by_airline['United']['processed_text'])
tfidf_df_real = pd.DataFrame(data = sparse_out2.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_real_by_airline['Delta'].index)


# X_real_join = X_real_by_airline['Delta'].drop(['processed_text', 'dirty_text'], axis=1)
# X_real_by_airline['Delta'].drop(['processed_text'], axis=1, inplace = True)
X_real_vec = X_real_by_airline['Delta'].drop(columns= ['processed_text', 'dirty_text', 'time_of_tweet']).join(tfidf_df_real)

In [452]:
preds = lrgrid3.best_estimator_.predict(X_real_vec)

In [453]:
results = pd.DataFrame(preds).join(X_real_by_airline['United'][['dirty_text','processed_text']]).rename(columns={0:'sentiment'})

In [455]:
results[results['sentiment'] == 'negative']

,sentiment,dirty_text,processed_text
1,negative,"Dear @united, why fly when you lose people’s bags? #unitedairlines lose bags and do nothing about it… @CNTraveler @cntraveller",dear why fly when you lose people bag unitedairlines lose bag and do nothing about it
3,negative,ICAO: #A4460D\nFlt: #UAL560 #UnitedAirlines #BOS-#IAH\nFirst seen: 2022/08/19 17:53:04\nMin Alt: 1800 ft MSL\nMin Dist:… https://t.co/IrCOiRM01A,icao anumberd flt ualnumber unitedairlines bosiah first see number number number number min alt number ft msl min dist tcoircoirmnumbera
7,negative,It was the most anticipated game.🤩🤩🤩 #AndroidGameplay #UnitedAirlines #browsergames #リーグオブエンジェルズ https://t.co/bBa81TkeCQ,it be the most anticipated game starstruck starstruck starstruck androidgameplay unitedairlines browsergames tcobbanumbertkecq
8,negative,ICAO: #A60814\nFlt: #UAL561 #UnitedAirlines #BOS-#ORD\nFirst seen: 2022/08/19 16:59:33\nMin Alt: 2050 ft MSL\nMin Dist:… https://t.co/wwCMc2TTjO,icao anumber flt ualnumber unitedairlines bosord first see number number number number min alt number ft msl min dist tcowwcmcnumberttjo
11,negative,ICAO: #A620B7\nFlt: #UAL426 #UnitedAirlines #BOS-#IAH\nFirst seen: 2022/08/19 16:34:44\nMin Alt: 3150 ft MSL\nMin Dist:… https://t.co/FrSVTdSU6k,icao anumberbnumber flt ualnumber unitedairlines bosiah first see number number number number min alt number ft msl min dist tcofrsvtdsunumberk
12,negative,ICAO: #A620B7\nFlt: #UAL426 #UnitedAirlines #BOS-#IAH\nFirst seen: 2022/08/19 16:34:07\nMin Alt: 2000 ft MSL\nMin Dist:… https://t.co/aUYrF1fvoJ,icao anumberbnumber flt ualnumber unitedairlines bosiah first see number number number number min alt number ft msl min dist tcoauyrfnumberfvoj
13,negative,RT @airwaysmagazine: United Airlines (UA) had played a crucial role in the development of the Boeing 767. #AviationHistory #Boeing #Boeing7…,rt unite airline ua have play a crucial role in the development of the boeing number aviationhistory boeing boeingnumber
14,negative,ICAO: AAF9EA\nFlt: UAL1945 #UnitedAirlines #SEA-#SFO\nFirst seen: 2022/08/19 20:07:49\nMin Alt: -6 ft AGL\nMin Dist: 0.… https://t.co/jOqD74UTLo,icao aafnumberea flt ualnumber unitedairlines seasfo first see number number number number min alt number ft agl min dist number tcojoqdnumberutlo
15,negative,It was the most anticipated game.🤩🤩🤩 #AndroidGameplay #UnitedAirlines #browsergames #リーグオブエンジェルズ https://t.co/0sdnhlXHe9,it be the most anticipated game starstruck starstruck starstruck androidgameplay unitedairlines browsergames tconumbersdnhlxhenumber
17,negative,ICAO: #A60814\nFlt: #UAL1057 #UnitedAirlines #DEN-#SFO\nFirst seen: 2022/08/19 15:30:26\nMin Alt: 4325 ft MSL\nMin Dist… https://t.co/Me728C6CKs,icao anumber flt ualnumber unitedairlines densfo first see number number number number min alt number ft msl min dist tcomenumbercnumbercks


In [431]:
pd.set_option('display.max_colwidth', None)

In [ ]:
process_text(search_results['statuses'][9]['text'])

In [325]:
import tweepy

# API keyws that yous saved earlier
api_key = "iAimqbAdWiBrUfXe0Xmuhlyps"
api_secrets = "YWUyAsX8sA2VuJCwQv4x3MUhAGmm48VlV0ZTx2yWNWqlGCP9Kj"
access_token = "54188780-kRmlqKRHleSb1WBBTfrj4CR57wLxJYuU5qH9piYlq"
access_secret = 'JXzf6VKWIFhVhAuIYWWQX04jCaHgfgjia77ARpK7v1Z3p'

# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key, api_secrets)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print('Successful Authentication')
except:
    print('Failed authentication')

Successful Authentication


In [326]:
search_words = "#wildfires"
date_since = "2022-11-16"

In [329]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(5)
tweets

AttributeError: 'API' object has no attribute 'search'